In [ ]:
#imorting neccessray modules
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
#loading the dataset
iris = datasets.load_iris()
data = pd.DataFrame(iris.data,columns = iris.feature_names)
print(data)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]


In [ ]:
target = iris.target_names #data of 3 different types of iris
labels = iris.target
print(target)
print()
print(labels) #labels/output

['setosa' 'versicolor' 'virginica']

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:
print("Before Normalization: ")
print(data)
print()

#normalization
# we are doing normalization to bring all feature values in a common range so that there is no missing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print("After Normalization")
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
print(data)

Before Normalization: 
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]

After Normalizat

In [ ]:
# converting data to numpy array for using it as datapoints/patterns for k-medoids algorithm
datapoints = data.to_numpy()
print(datapoints)
m, f = datapoints.shape

print()
print("No of patterns: " ,m,"No of Features: " , f)
k = 3 # no of clusters

[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]
 [0.30555556 0.79166667 0.11864407 0.125     ]
 [0.08333333 0.58333333 0.06779661 0.08333333]
 [0.19444444 0.58333333 0.08474576 0.04166667]
 [0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.45833333 0.08474576 0.        ]
 [0.30555556 0.70833333 0.08474576 0.04166667]
 [0.13888889 0.58333333 0.10169492 0.04166667]
 [0.13888889 0.41666667 0.06779661 0.        ]
 [0.         0.41666667 0.01694915 0.        ]
 [0.41666667 0.83333333 0.03389831 0.04166667]
 [0.38888889 1.         0.08474576 0.125     ]
 [0.30555556 0.79166667 0.05084746 0.125     ]
 [0.22222222 0.625      0.06779661 0.08333333]
 [0.38888889 0.75       0.11864407 0.08333333]
 [0.22222222 0.75       0.08474576 0.08333333]
 [0.30555556 0.58333333 0.11864407 0.04166667]
 [0.22222222 

In [ ]:
#randomly select 𝑘 of the 𝑚 data points as the medoids
#It should return a NumPy array of size 𝑘×𝑑, where 𝑑 is the number of columns of X.

def init_medoids(X, k):
    from numpy.random import choice
    from numpy.random import seed
 
    seed(1)
    samples = choice(len(X), size=k, replace=False)
    return X[samples, :]

medoids_initial = init_medoids(datapoints, 3)
print("Intial Medoids")
print(medoids_initial)

Intial Medoids
[[0.41666667 0.83333333 0.03389831 0.04166667]
 [0.22222222 0.20833333 0.33898305 0.41666667]
 [0.63888889 0.41666667 0.57627119 0.54166667]]


In [ ]:
# it will compute distances distance from an object/pattern to k medoids/representatives respectively
# it will return a numpy array of array of size m X 𝑘, where m is the number of rows of X.


def compute_d_p(X, medoids):
    m = len(X)
    medoids_shape = medoids.shape
    # If a 1-D array is provided, 
    # it will be reshaped to a single row 2-D array
    if len(medoids_shape) == 1: 
        medoids = medoids.reshape((1,len(medoids)))
    k = len(medoids)
    
    S = np.empty((m, k))
    
    for i in range(m):
        d_i = np.linalg.norm(X[i, :] - medoids, axis=1) #calculating eucliden distance using inbuilt function
        S[i, :] = d_i  #storing distance in numpy array S

    return S

In [ ]:
# compute numpy array of distances between object(pattern) to k medoids(cluster representative)
#this numpy array will contain k distances(d1,d2...dk)
#d1:distance between pattern1 to medoid 1(represetative of cluster 1)
#d2:distance between pattern2 to medoid 1(represetative of cluster 2)

# we will use this distances for cluster assignment

S = compute_d_p(datapoints, medoids_initial)
print(S)

[[0.28698522 0.62271839 0.85179826]
 [0.48709363 0.51054508 0.85530125]
 [0.452507   0.56662355 0.90086843]
 [0.50430306 0.53577208 0.89553075]
 [0.27983851 0.65192648 0.87668538]
 [0.16795261 0.69342695 0.79672442]
 [0.42011465 0.58699942 0.89723776]
 [0.33833146 0.58877687 0.84670295]
 [0.60204079 0.5289173  0.94007409]
 [0.45546303 0.55121151 0.8716249 ]
 [0.17480315 0.67985737 0.8293237 ]
 [0.37981163 0.58694131 0.86775597]
 [0.50364354 0.54543599 0.89551615]
 [0.59097006 0.60836045 1.00718565]
 [0.         0.81371699 0.87587794]
 [0.19513916 0.8967831  0.90442586]
 [0.14599147 0.7178536  0.83750661]
 [0.28999418 0.59855184 0.82802761]
 [0.12897288 0.69342695 0.77013189]
 [0.22152802 0.68494561 0.85813106]
 [0.28640448 0.58029623 0.77350596]
 [0.25092539 0.6322231  0.82090737]
 [0.37421649 0.69634336 0.97633522]
 [0.38168695 0.47133892 0.73435937]
 [0.39209298 0.56829092 0.8410255 ]
 [0.47706415 0.49102448 0.82021552]
 [0.35131687 0.53176283 0.79002683]
 [0.271599   0.61615283 0.82

In [ ]:
#cluster assignment
#assigning  each pattern of X to that cluster which have minimum distance from all of the k-medoids

#function that acts on the distance matrix S to assign a “cluster label” of 0, 1, 2 to each point using the minimum distance to find the “most similar” medoid.
def assign_labels(S):
    return np.argmin(S, axis=1)

In [ ]:
#cluster assignment
labels = assign_labels(S)
#each Patterns belong to which cluster
print(labels)

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 0 0 0 0 2 2 2 1 2 2 2 1 2 1 1 2 1 2 1 2 2 1 2 1 2 2 2 2
 2 2 2 2 2 1 1 1 1 2 2 2 2 2 2 1 1 2 1 1 1 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:
# medoids Updation
# for each medoid 𝑗 and each data point 𝑖 associated to 𝑗 swap 𝑗 and 𝑖 and compute the total cost of the configuration (that is the average dissimilarity of 𝑖 to all the data points)
# Select the medoid 𝑗 with the lowest cost of the configuration.

def update_medoids(X, medoids):
    
    S = compute_d_p(datapoints, medoids) #computes distance matrix
    labels = assign_labels(S) #cluster assignment on the basis of distance matix
        
    out_medoids = medoids  #out_medoids will store the updated medoids
                
    
    #for each pair of selected medoid i
    for i in set(labels):
        
        #curr SSE
        avg_dissimilarity = np.sum(compute_d_p(datapoints, medoids[i]))

        #and non-selected medoid j of that cluster
        cluster_points = datapoints[labels == i]
       
        #for each cluster, search if any of the points in the cluster decreases the average dissimilarity coefficient.
        #Select the point that decreases this coefficient the most as the new medoid for this cluster.
        for datap in cluster_points:
            new_medoid = datap

            #new SSE
            new_dissimilarity= np.sum(compute_d_p(datapoints, datap))
            
            if new_dissimilarity < avg_dissimilarity :
                avg_dissimilarity = new_dissimilarity #update SSE for that cluster
                
                out_medoids[i] = datap #update the medoid of that cluster
                
    return out_medoids

In [ ]:
# This is the function to check whether the medoids no longer move and the iteration should be stopped.
def has_converged(old_medoids, medoids):
    return set([tuple(x) for x in old_medoids]) == set([tuple(x) for x in medoids])

In [ ]:
# Main Algorithm
#X:Datapoints , k:no of clusters ,max_steps: no of iterations

def kmedoids(X, k, starting_medoids=None, max_steps=np.inf):
    
    #intialising the medoids if not present
    if starting_medoids is None:
        medoids = init_medoids(X, k)
    else:
        medoids = starting_medoids
        
    
    converged = False #for convergence checking
    labels = np.zeros(len(X))
    i = 1

    #convergence criteria
    while (not converged) and (i <= max_steps):
       
        old_medoids = medoids.copy()   #old medoids
        
        S = compute_d_p(X, medoids)   #computes distance matrix
        
        labels = assign_labels(S)     #cluster assignment on the basis of distance matrix
        
        medoids = update_medoids(X, medoids)  #updation of medoids to find best set of medoids(i.e which have smallest sse value)
        
        converged = has_converged(old_medoids, medoids) #convergence checking
        i += 1
    return (medoids,labels)   #return final set of medoids and cluster assignments



In [ ]:
# calling k-medoids function
results = kmedoids(datapoints, 3)
final_medoids = results[0]

print()
print("Final Medoids Set")
print(final_medoids)
print()
print("Final Clusters")
data['clusters'] = results[1]
print(data['clusters'])



Final Medoids Set
[[0.22222222 0.54166667 0.11864407 0.16666667]
 [0.44444444 0.41666667 0.54237288 0.58333333]
 [0.38888889 0.375      0.54237288 0.5       ]]

Final Clusters
0      0
1      0
2      0
3      0
4      0
      ..
145    1
146    1
147    1
148    1
149    1
Name: clusters, Length: 150, dtype: int64


In [ ]:
#computing Silhouette_score and sse
score = silhouette_score(datapoints, labels, metric='euclidean')
print("Silhouette coefficient:", score)
sse = np.sum(compute_d_p(datapoints, final_medoids))
print("SSE:", sse)

Silhouette coefficient: 0.4540446392851113
SSE: 239.5336042398771


In [ ]:
#using inbuilt k-means
kmeans = KMeans(n_clusters=3, random_state=0).fit(data)

silhouette_kmeans = silhouette_score(data, kmeans.predict(data))
print(" Silhoutte score:",silhouette_kmeans)

 Silhoutte score: 0.7657042112393824


Own implementation of k-means

In [ ]:
#Using my own implementation for k-means





#importing necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import random
import math
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
#computes euclidean distance between x and y
def euclidianDistance( x, y):
  return np.sqrt(np.sum(np.power((x - y), 2)))

In [ ]:
# X: dataset without label column and it contains 4 features and 150 patterns
# k: number of clusters
# mx_itr : maximum no of iterations/ convergence criteria of kMeans
def k_means_fit(X,k,mx_itr=300):
  centers = {}

##step 1: randomly choose k objects from dataset X as intial cluster centers
  for i in range(k):
    centers[i] = X[i]
  
   #step 4:check for convergence
  for i in range(mx_itr):
    curr_classification = {}  # stores the k clusters
    distance = []    # stores distance between curr obj and k cluster centers
    
    for j in range(k):
      curr_classification[j] = []

    #step 2: for each objects in X do some operations:
    for row in X:
      #1: compute distance between curr obj and k cluster centers
      distance = [euclidianDistance(row,centers[center]) for center in centers]
      
      #2: assign the object to that cluster to which it is closest by finding the index of minimum distance cluster center
      curr_class = distance.index(min(distance))
      curr_classification[curr_class].append(row)

    #step 3: update the cluster centers by taking the average of the patterns that belong to that cluster
    for j in range(k):  
      centers[j] = np.average(curr_classification[j],axis=0)  
    
  return centers



In [ ]:
#classify each object to its closest cluster
# predicts cluster of each objects : means objects belong to which cluster

def predict(X,centers):
  cls =[]
  for row in X:
    distance = [euclidianDistance(row,centers[center]) for center in centers]
    curr_class = distance.index(min(distance))
    cls.append(curr_class)
  return np.array(cls)

In [ ]:
#loading the iris dataset
iris = datasets.load_iris()      

df = pd.DataFrame(iris.data)
df.columns = iris.feature_names

df['target'] = iris.target

X = df.iloc[:,0:4].to_numpy()               
Y = df.iloc[:,-1].to_numpy() 

In [ ]:
center = k_means_fit(X,3)  #finding k cluster centers after k-means operation
center
pred_class = predict(X,center) ##classify each object to its closest cluster

silhouette_kmeans = silhouette_score(X, pred_class) #computes silhouette score
print(" Silhoutte score:",silhouette_kmeans)

 Silhoutte score: 0.5511916046195916
